In [12]:
import pandas as pd
from pandas import json_normalize
import requests
import json

In [13]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
import folium
from pandas import json_normalize
import requests
import json

In [24]:
geolocator = Nominatim(timeout=10, user_agent="PDS")

df_geocode = pd.read_csv('../dataset_homegate_enriched.csv', sep=',', encoding='utf-8')[['web-scraper-order', 'Street', 'full_address', 'PLZ', 'Ortschaftsname', 'Kantonskürzel']]
df_geocode.head(5)

,web-scraper-order,Street,full_address,PLZ,Ortschaftsname,Kantonskürzel
0,1672673926-34302,,6963 Pregassona,6963,Pregassona,TI
1,1672673926-34301,Via Ceresio 39A,"Via Ceresio 39A, 6963 Pregassona",6963,Pregassona,TI
2,1672673272-29951,Via Vedo 3,"Via Vedo 3, 6963 Cureggia",6963,Pregassona,TI
3,1672673272-29952,Via vedo 3,"Via vedo 3, 6963 Cureggia",6963,Pregassona,TI
4,1672673816-33571,via Ceresio,"via Ceresio, 6963 Pregassona",6963,Pregassona,TI


In [25]:
housenumber = []
for i in df_geocode['Street']:
    
    try:
        d1 = re.findall('\d+', i)

        d2 = d1[0].strip().replace(',', '.')
    except:
        d2 = None
    housenumber.append(d2)

# Save as new variable in the pandas data frame
df_geocode['housenumber'] = pd.Series(housenumber, dtype="float64")
    
# Print first 5 values
print(df_geocode['Street'].head(5), '\n')
print(df_geocode['housenumber'].head(5))

0                   
1    Via Ceresio 39A
2         Via Vedo 3
3         Via vedo 3
4        via Ceresio
Name: Street, dtype: object 

0     NaN
1    39.0
2     3.0
3     3.0
4     NaN
Name: housenumber, dtype: float64


In [26]:
df_geocode.head(5)

,web-scraper-order,Street,full_address,PLZ,Ortschaftsname,Kantonskürzel,housenumber
0,1672673926-34302,,6963 Pregassona,6963,Pregassona,TI,NaN
1,1672673926-34301,Via Ceresio 39A,"Via Ceresio 39A, 6963 Pregassona",6963,Pregassona,TI,39.0
2,1672673272-29951,Via Vedo 3,"Via Vedo 3, 6963 Cureggia",6963,Pregassona,TI,3.0
3,1672673272-29952,Via vedo 3,"Via vedo 3, 6963 Cureggia",6963,Pregassona,TI,3.0
4,1672673816-33571,via Ceresio,"via Ceresio, 6963 Pregassona",6963,Pregassona,TI,NaN


In [27]:
# Count missing values
print('Count missing values per variable')
print (pd.isna(df_geocode).sum(), '\n')

# Identify row with missing values
print('Identify rows with missing values')
print(df_geocode.loc[df_geocode.isna().any(axis=1)][['Street', 'housenumber']], '\n')


# Drop rows where at least one element is missing.
df_geocode = df_geocode.dropna(subset=['Street', 'housenumber'])  

df_geocode.drop(df_geocode[df_geocode['Street'] == " "].index, inplace=True)
df_geocode.head(5)

Count missing values per variable
web-scraper-order       0
Street                  0
full_address            0
PLZ                     0
Ortschaftsname          0
Kantonskürzel           0
housenumber          8352
dtype: int64 

Identify rows with missing values
            Street  housenumber
0                           NaN
4      via Ceresio          NaN
5                           NaN
6                           NaN
7                           NaN
...            ...          ...
11296     La Motta          NaN
11297                       NaN
11299                       NaN
11300                       NaN
11301                       NaN

[8352 rows x 2 columns] 



,web-scraper-order,Street,full_address,PLZ,Ortschaftsname,Kantonskürzel,housenumber
1,1672673926-34301,Via Ceresio 39A,"Via Ceresio 39A, 6963 Pregassona",6963,Pregassona,TI,39.0
2,1672673272-29951,Via Vedo 3,"Via Vedo 3, 6963 Cureggia",6963,Pregassona,TI,3.0
3,1672673272-29952,Via vedo 3,"Via vedo 3, 6963 Cureggia",6963,Pregassona,TI,3.0
18,1672673941-34412,Via Aprica 18,"Via Aprica 18, 6900 Lugano",6900,Lugano,TI,18.0
20,1672673938-34399,Via al Roccolo 6,"Via al Roccolo 6, 6900 Massagno",6900,Lugano,TI,6.0


In [28]:
#Dimension (rows, columns)
print('Dimension: ', df_geocode.shape)

#Number of Rows
print('Number of Rows: ', df_geocode.shape[0])

#Number of Columns
print('Number of Columns: ', df_geocode.shape[1])

Dimension:  (2950, 7)
Number of Rows:  2950
Number of Columns:  7


In [29]:
strplz = []
for i in df_geocode['PLZ']:
    intplz = i
    strplz.append(str(i))

df_geocode['strplz'] = pd.Series(strplz, dtype="str")

address = df_geocode['Street'] + " " + df_geocode['Ortschaftsname']

df_geocode['address_geocoding'] = pd.Series(address, dtype="str")

df_geocode.head(5)

,web-scraper-order,Street,full_address,PLZ,Ortschaftsname,Kantonskürzel,housenumber,strplz,address_geocoding
1,1672673926-34301,Via Ceresio 39A,"Via Ceresio 39A, 6963 Pregassona",6963,Pregassona,TI,39.0,6963,Via Ceresio 39A Pregassona
2,1672673272-29951,Via Vedo 3,"Via Vedo 3, 6963 Cureggia",6963,Pregassona,TI,3.0,6963,Via Vedo 3 Pregassona
3,1672673272-29952,Via vedo 3,"Via vedo 3, 6963 Cureggia",6963,Pregassona,TI,3.0,6900,Via vedo 3 Pregassona
18,1672673941-34412,Via Aprica 18,"Via Aprica 18, 6900 Lugano",6900,Lugano,TI,18.0,6965,Via Aprica 18 Lugano
20,1672673938-34399,Via al Roccolo 6,"Via al Roccolo 6, 6900 Massagno",6900,Lugano,TI,6.0,6914,Via al Roccolo 6 Lugano


In [30]:
import urllib
from IPython.display import clear_output
# Define base url
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Geocode list of adresses
geolocation = []
n = 1
for i in df_geocode['address_geocoding'].astype(str):
    
    print('Geocoding address', 
          n, 
          'out of', 
          len(df_geocode['address_geocoding']), 
          ':', 
          i)
    n=n+1
    clear_output(wait=True)
    
    try:
        # Set up search parameters - address, origins and type
        parameters = {"searchText": i,
                      "origins": "address",
                      "type": "locations",
                     }

        # Server request
        r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

        # Get data
        data = json.loads(r.content)

        # Take first server response, convert to df with relevant infos
        df_loc = pd.DataFrame.from_dict(list(data.values())[0][0], 
                                        orient='columns')
        geolocation.append(df_loc.iloc[[5,6],0].astype(float))
    
    except:
        geolocation.append(pd.Series(data={'lat': None, 'lon': None}))
        
        

# Write lat and lon to df
df_loc = pd.DataFrame(geolocation, 
                      columns=("lat", "lon"), 
                      index=range(len(df_geocode['address_geocoding'])))
df_geocode['latitude'] = df_loc['lat']
df_geocode['longitude'] = df_loc['lon']
df_geocode.head(5)

,web-scraper-order,Street,full_address,PLZ,Ortschaftsname,Kantonskürzel,housenumber,strplz,address_geocoding,latitude,longitude
1,1672673926-34301,Via Ceresio 39A,"Via Ceresio 39A, 6963 Pregassona",6963,Pregassona,TI,39.0,6963,Via Ceresio 39A Pregassona,46.021164,8.973758
2,1672673272-29951,Via Vedo 3,"Via Vedo 3, 6963 Cureggia",6963,Pregassona,TI,3.0,6963,Via Vedo 3 Pregassona,46.021164,8.973758
3,1672673272-29952,Via vedo 3,"Via vedo 3, 6963 Cureggia",6963,Pregassona,TI,3.0,6900,Via vedo 3 Pregassona,46.001793,8.944041
18,1672673941-34412,Via Aprica 18,"Via Aprica 18, 6900 Lugano",6900,Lugano,TI,18.0,6965,Via Aprica 18 Lugano,46.050957,8.988699
20,1672673938-34399,Via al Roccolo 6,"Via al Roccolo 6, 6900 Massagno",6900,Lugano,TI,6.0,6914,Via al Roccolo 6 Lugano,NaN,NaN


In [11]:
'''
import geocoder

latitude = []
longitude = []

for address in df_geocode['address_geocoding']:
    g = geocoder.arcgis(address)
    longitude.append(g.lng)
    latitude.append(g.lat)
    print(g.latlng)

    df_geocode['latitude'] = pd.Series(latitude, dtype="Float32")
    df_geocode['longitude'] = pd.Series(longitude, dtype="Float32")
df_geocode.head(5)
'''

'\nimport geocoder\n\nlatitude = []\nlongitude = []\n\nfor address in df_geocode[\'address_geocoding\']:\n    g = geocoder.arcgis(address)\n    longitude.append(g.lng)\n    latitude.append(g.lat)\n    print(g.latlng)\n\n    df_geocode[\'latitude\'] = pd.Series(latitude, dtype="Float32")\n    df_geocode[\'longitude\'] = pd.Series(longitude, dtype="Float32")\ndf_geocode.head(5)\n'

In [31]:
# Count missing values
print('Count missing values per variable')
print (pd.isna(df_geocode).sum(), '\n')

# Identify row with missing values
print('Identify rows with missing values')
print(df_geocode.loc[df_geocode.isna().any(axis=1)][['longitude', 'latitude']], '\n')

df_geocode = df_geocode.dropna(subset=['longitude', 'latitude'])  
df_geocode.head(5)


Count missing values per variable
web-scraper-order       0
Street                  0
full_address            0
PLZ                     0
Ortschaftsname          0
Kantonskürzel           0
housenumber             0
strplz               2385
address_geocoding       0
latitude             2487
longitude            2487
dtype: int64 

Identify rows with missing values
       longitude  latitude
20           NaN       NaN
407          NaN       NaN
497          NaN       NaN
570          NaN       NaN
573          NaN       NaN
...          ...       ...
11283        NaN       NaN
11284        NaN       NaN
11290        NaN       NaN
11291        NaN       NaN
11298        NaN       NaN

[2487 rows x 2 columns] 



,web-scraper-order,Street,full_address,PLZ,Ortschaftsname,Kantonskürzel,housenumber,strplz,address_geocoding,latitude,longitude
1,1672673926-34301,Via Ceresio 39A,"Via Ceresio 39A, 6963 Pregassona",6963,Pregassona,TI,39.0,6963,Via Ceresio 39A Pregassona,46.021164,8.973758
2,1672673272-29951,Via Vedo 3,"Via Vedo 3, 6963 Cureggia",6963,Pregassona,TI,3.0,6963,Via Vedo 3 Pregassona,46.021164,8.973758
3,1672673272-29952,Via vedo 3,"Via vedo 3, 6963 Cureggia",6963,Pregassona,TI,3.0,6900,Via vedo 3 Pregassona,46.001793,8.944041
18,1672673941-34412,Via Aprica 18,"Via Aprica 18, 6900 Lugano",6900,Lugano,TI,18.0,6965,Via Aprica 18 Lugano,46.050957,8.988699
32,1672673938-34382,Riva Caccia 1A,"Riva Caccia 1A, 6900 Lugano",6900,Lugano,TI,1.0,6932,Riva Caccia 1A Lugano,46.009281,8.925307


In [34]:
df_geocode.to_csv('dataset_addresses_geocoded.csv',
                      sep=",",
                      encoding='utf-8',
                      index=False)

In [33]:
# Subset of appartments
locations = df_geocode[["latitude", "longitude", "Ortschaftsname"]]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.latitude.mean(), 
                           locations.longitude.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["latitude"], 
                   location_info["longitude"]], 
                  popup=location_info["Ortschaftsname"]).add_to(map)

# Plot map
map

     latitude  longitude Ortschaftsname
1   46.021164   8.973758     Pregassona
2   46.021164   8.973758     Pregassona
3   46.001793   8.944041     Pregassona
18  46.050957   8.988699         Lugano
32  46.009281   8.925307         Lugano
